In [ ]:
import mbuild as mb
import foyer
import unyt as u

## Prepare recipes with mBuild

### Exercise 1

In [ ]:
class _CH2(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH2"
        """Create a partilce name _CH2 and add it to self."""
        # Enter your code here
        
        
        """Create a port attached to the _CH2 particles, label as 'up'."""
        port_up = mb.Port(anchor=_ch2, orientation=[0, 0, 1], separation=0.07)
        self.add(port_up, "up")
        
        
        """Create a port attached to the _CH2 particles, label as 'down',
        with same separation but opposite orientation as 'up port'
        """
        # Enter your code here

        
class _CH3(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH3"
        
        """Create a partilce name _CH3 and add it to self."""
        # Enter your code here
        
        
        """Create a port attached to the _CH3 particles, label as 'up'.
        Use the separation value of the _CH2's ports as above.
        """

            

### Exercise 2a

In [ ]:
class CG_Alkane(mb.Compound):
    def __init__(self, n):
        super().__init__()
        self.name = "CG_Alkane"
    
        if n == 1: 
            """Create a _CH4 bead and add it to self."""
        elif n == 2:
            """Create 2 _CH3 and bonds them together, then add both to self."""
        else:
            """ Create a _CH3 particle and add it to self with label 'link0'"""
            # Enter your code here

            
            for i in range(n - 2):
                """ Create a _CH2 particle and add it to self with label f'link{i}'.
                Then force_overlap it (from Port "down") with 
                the previous link (to Port "up").
                """
                
                
            """ Create another _CH3 particle and add it to self with label f'link{i}.
            Then force_overlap it with the previous link.
            """
            # Enter your code here
            


### Exercise 2b

In [ ]:
from mbuild.lib.recipes import Polymer

help(Polymer)
"""Create a Polymer() object with _CH2 monomer and ends with 2 _CH3."""
# Enter your code here

"""Build the just-created object with length 5"""
# Enter your code here


### Exercise 3

In [ ]:
"""Create a pentane using the CG_Alkane recipe above."""
# Enter your code here

"""Create a box of pentane using the mb.packing.fill_box() method.
Let's fill the box of size [3.2, 3.2, 3.2] with a padding of 0.2 
with 150 compounds."""
# Enter your code here

"""Perform a brief energy minimization of the pentane box created above
with the trappe forcefield"""
# Enter your code here
help(mb.Compound.energy_minimize)

## Atomtyping and Parameterization with Foyer
### Exercise 4

In [ ]:
import foyer 
from foyer import Forcefield

"""Load the TRAPPE-UA forcefield (name "trappe-ua")"""
# Enter your code here

"""Use foyer to apply forcefields to a pentane and a box of pentane"""
# Enter your code here

### Exercise 5a - Simulation with MosDeF Cassandra

In [ ]:
import mosdef_cassandra as mc

%mkdir mc_sim
%cd mc_sim
# Create box and species list
species_list = [typed_pentane]
box_list = [pentane_box]
mols_in_boxes = [[150]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moveset = mc.MoveSet("npt", species_list)

moveset.prob_volume = 0.010

thermo_props = [
    "energy_total",
    "energy_intervdw",
    "pressure",
    "volume",
    "nmols",
    "mass_density",
]

custom_args = {
    "run_name": "equil",
    "charge_style": "none",
    "rcut_min": 2.0 * u.angstrom,
    "vdw_cutoff": 14.0 * u.angstrom,
    "units": "sweeps",
    "steps_per_sweep": 450,
    "coord_freq": 50,
    "prop_freq": 10,
    "properties": thermo_props,
}


mc.run(
    system=system,
    moveset=moveset,
    run_type="equilibration",
    run_length=2400,
    pressure=14.02 * u.bar,
    temperature=372 * u.K,
    **custom_args,
)
%cd ..

### Exercise 5b - Simulation with GROMACS

In [ ]:
em_mdp = """
integrator          = steep
nsteps              = 500000
emstep              = 0.002
emtol               = 10
dt                  = 0.002

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4

gen_vel             = yes
gen-temp            = 372.0
gen-seed            = 4

tcoupl              = no

pcoupl              = no

pbc                 = xyz

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

nvt_mdp = """
integrator          = md
nsteps              = 1000000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1
ref_t               = 372.0

pcoupl              = no

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

npt_mdp = """
integrator          = md
nsteps              = 1000000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 1000
nstvout             = 1000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz 

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4 

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4 

gen_vel             = no

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1 
ref_t               = 372.0

pcoupl                   = parrinello-rahman
pcoupltype               = isotropic
nstpcouple               = -1
tau-p                    = 10.0
compressibility          = 4.5e-5
ref-p                    = 14.02

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

%mkdir gmx_sim
with open("gmx_sim/em.mdp", "w") as f:
    f.write(em_mdp)

with open("gmx_sim/nvt.mdp", "w") as f:
    f.write(nvt_mdp)

with open("gmx_sim/npt.mdp", "w") as f:
    f.write(npt_mdp)

In [ ]:
#Save out the typed system above to init.gro and inti.top file in the gmx_sim dir

In [ ]:
%cd gmx_sim
!gmx grompp -f em.mdp -o em.tpr -c init.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt

!gmx grompp -f nvt.mdp -o nvt.tpr -c em.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt

!gmx grompp -f npt.mdp -o npt.tpr -c nvt.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm npt -s npt.tpr -cpi npt.cpt
%cd ..

### Exercise 5c - Simulation with HOOMD-Blue

In [ ]:
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import hoomd 

# ref_distance: 10 angstrom -> 1 nm
# ref_energy: 1/4.184 kcal/mol -> 1 kJ/mol
# ref_mass: 0.9999938574 dalton -> 1 amu
d = 10
e = 1 / 4.184
m = 0.9999938574

"""Converting a typed structure to hoomd forcefield and snapshot"""
snapshot, forcefield, ref_vals = create_hoomd_forcefield(
    structure,
    ref_distance=d,
    ref_energy=e,
    ref_mass=m,
    r_cut=1,
    init_snap=None,
    pppm_kwargs={"Nx": 64, "Ny": 64, "Nz": 64, "order": 7},
)

for force in forcefield:
    if isinstance(force, hoomd.md.pair.LJ):
        force.tail_correction = True
        
forcefield[0].nlist.exclusions = ["bond", "1-3", "1-4"]

"""Starting up the system and adding in operations and loggers"""
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device=device)
sim.create_state_from_snapshot(snapshot)

gsd_writer = hoomd.write.GSD(
    filename=f"hoomd_sim/trajectory.gsd",
    trigger=hoomd.trigger.Periodic(10000),
    mode="wb",
    dynamic=["momentum"],
)
sim.operations.writers.append(gsd_writer)

logger = hoomd.logging.Logger(categories=["scalar", "string"])
logger.add(sim, quantities=["timestep", "tps"])
thermo_props = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo_props)
logger.add(
    thermo_props,
    quantities=[
        "kinetic_energy",
        "potential_energy",
        "pressure",
        "kinetic_temperature",
        "volume",
    ],
)

table_file = hoomd.write.Table(
    output=open(
        "hoomd_sim/log.txt", mode="w", newline="\n"
    ),
    trigger=hoomd.trigger.Periodic(period=1000),
    logger=logger,
    max_header_len=7,
)
sim.operations.writers.append(table_file)

"""Create integrators and add them to the simulation context, sequentially"""
dt = 0.001
integrator = hoomd.md.Integrator(dt=dt)
integrator.forces = forcefield

kT = (372 * u.K).to_equivalent("kJ/mol", "thermal").value

tau = 100 * dt 
tauS = 1000 * dt 

pressure = (14.02 * u.kPa).to("kJ/(mol*nm**3)").value

"""Run NVT"""
nvt_steps = 1e6

nvt_method = hoomd.md.methods.NVT(filter=hoomd.filter.All(),
                                         kT=kT,
                                         tau=tau)

integrator.methods = [nvt_method]

sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), 
                                      kT=kT)

sim.run(nvt_steps)

"""Run NPT"""
npt_steps = 1e6
npt_method = hoomd.md.methods.NPT(filter=hoomd.filter.All(),
                                 kT=kT, 
                                 tau=tau,
                                 S=pressure,
                                 tauS=tauS,
                                 couple="xyz")
integrator.methods = [npt_method]
sim.run(1e6)

### Exercise 6a - Data analysis of Cassandra simulation

In [ ]:
"""If you used mosdef cassandra"""
import numpy as np
import pylab as plt

"""In the interest of time, loading in pre-simulated data"""
data = np.genfromtxt("../master_notebooks/example_workflow/mc_sim/equil.out.prp", skip_header=3)

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r'MC sweeps')
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

ax.plot(data[:, 0], data[:,-1], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()

### Exercise 6b - Data analysis of GROMACS simulation

In [ ]:
"""If you used GROMACS"""
import numpy as np
import pylab as plt 

import panedr
from panedr import edr_to_df

"""In the interest of time, loading in pre-simulated data"""
data = edr_to_df("../master_notebooks/example_workflow/gmx_sim/npt.edr")

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r"MD Step")
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

dt, density = list(), list()
for i, j in enumerate(data["Density"]):
    dt.append(i)
    density.append(j)
    
ax.plot(dt, density, "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()

### Exercise 6c - Data analysis of HOOMD-Blue simulation


In [ ]:
import numpy as np
import pylab as plt 

"""In the interest of time, loading in pre-simulated data"""
data = np.genfromtxt("../master_notebooks/example_workflow/hoomd_sim/log.txt", names=True)
system_mass = 72.15 * u.amu * 150 
volume = data["volume"] * u.nm**3 

density = (system_mass / volume).to("kg/m**3")
plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r"MD Step")
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

    
ax.plot(data["timestep"][1000:], density[1000:], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()